In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from imutils import printImage, generateRadomColor, rotate

In [2]:
image1 = cv2.imread("./data/img1.jpg")
image2 = cv2.imread("./data/img2.jpg")

image3 = cv2.absdiff(image1, image2)

In [29]:
printImage(image3)

In [6]:
image3.shape

(720, 1280, 3)

In [7]:
cv2.imwrite("./data/eu.jpg", image3)

True

# Manipulando pixels

In [8]:
(b, g, r) = image3[0, 0]
print("Pixel at (0, 0) - Red: {}, Green: {}, Blue: {}".format(r, g, b))

Pixel at (0, 0) - Red: 7, Green: 10, Blue: 5


In [9]:
image3[0, 0] = (0, 0, 255)
(b, g, r) = image3[0, 0]
print("Pixel at (0, 0) - Red: {}, Green: {}, Blue: {}".format(r, g, b))

Pixel at (0, 0) - Red: 255, Green: 0, Blue: 0


In [30]:
for x in range(10000):
    i = random.randint(0, image3.shape[0] - 1)
    j = random.randint(0, image3.shape[1] - 1)
    image3[i, j] = generateRadomColor()

In [31]:
printImage(image3)

# Desenhando com OpenCV

In [12]:
import cv2
import numpy as np

canvas = np.zeros((300, 300, 3), dtype="uint8")

In [13]:
printImage(canvas)

### Linha

In [14]:
green = (0, 255, 0)
result = cv2.line(canvas, (0, 0), (300, 300), green, 5)
printImage(result)

### Retângulo

In [15]:
red = (0, 0, 255)
result = cv2.rectangle(canvas, (50, 200), (200, 225), red, 5)
printImage(result)

### Círculo

In [16]:
canvas = np.zeros((300, 300, 3), dtype="uint8")
(centerX, centerY) = (canvas.shape[1] // 2, canvas.shape[0] // 2)
white = (255, 255, 255)
for r in range(0, 175, 25):
    cv2.circle(canvas, (centerX, centerY), r, white)
printImage(canvas)

# Transformando Imagens

### Movendo a imagem dentro do frame

In [26]:
def translate(image, x, y):
    M = np.float32([[1, 0, x], [0, 1, y]]) # matriz de transformação
    shifted = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
    return shifted

In [25]:
x = 100 # shifting the image 100 pixels to the right
y = 50 # shifting the image 50 pixels down side
shifted = translate(image3, x, y)
printImage(shifted)

### Rotacionando a Imagem

In [8]:
(h, w) = image3.shape[:2]
center = (w // 2, h // 2)

M = cv2.getRotationMatrix2D(center, 45, 1.0) # rotacionando 45 graus
rotated = cv2.warpAffine(image3, M, (w, h))
printImage(rotated)

In [9]:
from imutils import rotate

rotated2 = rotate(image3, -90, scale=0.5)
printImage(rotated2)

### Redimensionando